# Train notebook - Airbus kaggle challenge

# Ship Detection- Project AIRBUS 2019
## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations)

In [1]:
BATCH_SIZE = 32
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING = (2,2)
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 10000
MAX_TRAIN_EPOCHS = 10

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen
from sklearn.model_selection import train_test_split
from keras import backend as K

Using TensorFlow backend.


In [3]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 15659486632036678130)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 1138902826440538480)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12779835977405274075)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 15560753152, 8802003206523597796)
Default GPU Device: /device:GPU:0


In [6]:
ship_dir = '/mnt/data/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [7]:
label_has_ships = masks
label_has_ships['has_ship'] = masks.EncodedPixels.notnull()
label_has_ships = label_has_ships.drop(columns=['EncodedPixels'])
label_has_ships = label_has_ships.drop_duplicates()

In [8]:
count_has_ship = label_has_ships.has_ship.sum()
count_has_ship
print("rate:", count_has_ship/len(label_has_ships))

rate: 0.2210058372629261


In [9]:
data_balanced = label_has_ships.groupby('has_ship')
data_balanced = data_balanced.apply(lambda x: x.sample(data_balanced.size().min()).reset_index(drop=True))
data_balanced

ImageId  has_ship
has_ship                               
False    0      02fca3ccb.jpg     False
         1      4150030ba.jpg     False
         2      af0b286c4.jpg     False
         3      d80eb19e2.jpg     False
         4      138612b34.jpg     False
         5      6ab37325d.jpg     False
         6      3d92ba4b7.jpg     False
         7      61a1e848f.jpg     False
         8      56db1176e.jpg     False
         9      ba2fe3a22.jpg     False
         10     a791a84e4.jpg     False
         11     5b05e7790.jpg     False
         12     fd6d95519.jpg     False
         13     edc047e6b.jpg     False
         14     42237e98f.jpg     False
         15     68eb24910.jpg     False
         16     46a4226cb.jpg     False
         17     52502801b.jpg     False
         18     286024f5c.jpg     False
         19     7692b1f16.jpg     False
         20     3f777da77.jpg     False
         21     3682a6790.jpg     False
         22     c3caab444.jpg     False
         23     5ba854bea.jpg     False
         24     84fa2fef2.jpg     False
         25     9a457e1b5.jpg     False
         26     34c22a38a.jpg     False
         27     594124904.jpg     False
         28     e7ec34332.jpg     False
         29     fce82aaad.jpg     False
...                       ...       ...
True     42526  e6fee96ed.jpg      True
         42527  92eed5c43.jpg      True
         42528  c1dc740c6.jpg      True
         42529  636bcf83e.jpg      True
         42530  58f8fb3e8.jpg      True
         42531  6d586c583.jpg      True
         42532  6b5d102cc.jpg      True
         42533  49f4f1063.jpg      True
         42534  f7cb73e26.jpg      True
         42535  1470678a5.jpg      True
         42536  c8057672f.jpg      True
         42537  861fec788.jpg      True
         42538  be4b05d86.jpg      True
         42539  213f9167f.jpg      True
         42540  f83a70bdc.jpg      True
         42541  6f34d01fa.jpg      True
         42542  181fb3009.jpg      True
         42543  fb193aa00.jpg      True
         42544  42afdde16.jpg      True
         42545  656e328c3.jpg      True
         42546  fb3101eba.jpg      True
         42547  089ee7570.jpg      True
         42548  e80a9a023.jpg      True
         42549  cdf12b81d.jpg      True
         42550  6ab8fd21a.jpg      True
         42551  4a0687163.jpg      True
         42552  df7f733b3.jpg      True
         42553  999c51378.jpg      True
         42554  d13d8a05c.jpg      True
         42555  91efd7814.jpg      True

[85112 rows x 2 columns]

In [10]:
count_has_ship = data_balanced.has_ship.sum()
count_has_ship
print("rate:", count_has_ship/len(data_balanced))

rate: 0.5


In [11]:
def make_image_gen(in_df, train_image_dir, batch_size=48, img_scalling=None):
    all_batches = list(in_df.groupby('ImageId'))
    out_rgb = []
    out_label = []
    while True:
        np.random.shuffle(all_batches)
        for c_img_id, c_label in all_batches:
            rgb_path = os.path.join(train_image_dir, c_img_id)
            c_img = imread(rgb_path)
            
            if img_scalling is not None:
                c_img = c_img[::img_scalling[0], ::img_scalling[1]]
                
            out_rgb.append(c_img)
            out_label.append(c_label.has_ship.values[0])
            if len(out_rgb)>=batch_size:
                yield np.stack(out_rgb, 0)/255.0, np.array(out_label)
                out_rgb, out_label=[], []   

In [12]:
training_set, validation_set = train_test_split(data_balanced, test_size=0.05)
print(len(training_set))
print(len(validation_set))

80856
4256


In [13]:
train_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING)
train_x, train_y = next(train_gen)
print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_x.min(), train_x.max())

x (32, 384, 384, 3) 0.0 1.0
y (32,) 0.0 1.0


In [14]:
train_y

array([False, False, False,  True, False,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True, False,  True, False,
       False, False, False, False, False, False,  True, False,  True,
       False,  True, False,  True,  True])

In [15]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from models.resnet50_classif import get_resnet50_classif

In [16]:
model = get_resnet50_classif(input_shape=train_x.shape[1:])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 4s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 384, 384, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 294912)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               37748864  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 61,336,705
Trainable params: 61,283,585
Non-trainable params: 53,120
_________________________________________________________________


In [17]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback
weight_path = "../weights_models/{}_weights.best.hdf5".format('model_1')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=2, verbose=1, mode='min',
                                   min_delta=0.001, cooldown=1, min_lr=1e-7)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=2) # probably needs to be more patient, but kaggle time is limited

tensorboard =  TensorBoard(log_dir="../logs/log1", update_freq='batch')

callbacks_list = [reduceLROnPlat, early]

In [18]:
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

model.compile(optimizer=Adam(), loss=binary_crossentropy, 
              metrics=["accuracy"])

step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//BATCH_SIZE)
training_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING)
#training_aug_gen =  create_aug_gen(training_gen, image_gen, label_gen)

step_count_valid = validation_set.shape[0]//BATCH_SIZE
validation_gen = make_image_gen(validation_set, train_image_dir, BATCH_SIZE, IMG_SCALING)

print("step_count_train =", step_count_train)
print("step_count_valid =", step_count_valid)


loss_history = [model.fit_generator(training_gen,
                                 steps_per_epoch=step_count_train,
                                 epochs=MAX_TRAIN_EPOCHS,
                                 callbacks=callbacks_list,
                                 validation_data=validation_gen,
                                 validation_steps=step_count_valid)]

step_count_train = 2526
step_count_valid = 133
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2526/2526 [==============================] - 2365s 936ms/step - loss: 0.9283 - acc: 0.6695 - val_loss: 1.0717 - val_acc: 0.6426
Epoch 2/10
2526/2526 [==============================] - 2217s 878ms/step - loss: 0.8786 - acc: 0.7122 - val_loss: 0.6150 - val_acc: 0.7540
Epoch 3/10
2526/2526 [==============================] - 2234s 884ms/step - loss: 0.6515 - acc: 0.7090 - val_loss: 0.4114 - val_acc: 0.8057
Epoch 4/10
2526/2526 [==============================] - 2231s 883ms/step - loss: 0.3665 - acc: 0.8270 - val_loss: 0.2698 - val_acc: 0.8863
Epoch 5/10
2526/2526 [==============================] - 2236s 885ms/step - loss: 0.2810 - acc: 0.8818 - val_loss: 0.2691 - val_acc: 0.9086
Epoch 6/10
2526/2526 [==============================] - 2231s 883ms/step - loss: 0.2324 - acc: 0.9069 - val_loss: 0.3260 - val_acc: 0.8966

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000

In [20]:
model.save("weights_models/{}_weights.best.hdf5".format('clf_95'))

In [ ]:
step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//BATCH_SIZE)
training_gen = make_image_gen(label_has_ships, train_image_dir, BATCH_SIZE, IMG_SCALING)
#training_aug_gen =  create_aug_gen(training_gen, image_gen, label_gen)

step_count_valid = validation_set.shape[0]//BATCH_SIZE
validation_gen = make_image_gen(validation_set, train_image_dir, BATCH_SIZE, IMG_SCALING)

print("step_count_train =", step_count_train)
print("step_count_valid =", step_count_valid)


loss_history = [model.fit_generator(training_gen,
                                 steps_per_epoch=step_count_train,
                                 epochs=5,
                                 callbacks=callbacks_list)]